In [1]:
import pandas as pd
import os
import sys
sys.path.append(os.path.pardir)
import core,model
import torch.optim as optim
import torch.nn.functional as F
import torch.nn as nn
import torch
import numpy as np
import pdb
import torch.tensor as tensor
from matplotlib import pyplot as plt
torch.cuda.set_device(1)

In [2]:
label_index_img_c = {'frog':0,'truck':1,'deer':2,'automobile':3,'bird':4,'horse':5,'ship':6,'cat':7,'airplane':8,'dog':9}
train_path = '/data/gpf/tutorial/dl/cnn/cifar/train'
test_path = '/data/gpf/tutorial/dl/cnn/cifar/test'

In [3]:
path_labels_train = core.get_img_path_label_from_path(train_path,label_index_img_c)

frog:5000
truck:5000
deer:5000
automobile:5000
bird:5000
horse:5000
ship:5000
cat:5000
airplane:5000
dog:5000


In [4]:
path_labels_test = core.get_img_path_label_from_path(test_path,label_index_img_c)

frog:1000
truck:1000
deer:1000
automobile:1000
bird:1000
horse:1000
ship:1000
cat:1000
airplane:1000
dog:1000


In [5]:
m = model.SimpleNet([32*32*3,40,10]).cuda()
m

SimpleNet(
  (layers): ModuleList(
    (0): Linear(in_features=3072, out_features=40, bias=True)
    (1): Linear(in_features=40, out_features=10, bias=True)
  )
)

In [18]:
opt = optim.SGD(m.parameters(),lr=0.01,momentum=0.9)
loss_fn = F.nll_loss

In [19]:
img_paths, labels = zip(*path_labels_train)
dl = core.dl_img(np.array(img_paths), np.array(labels).astype(int),bs=64)
core.fit(m,dl,30,opt,loss_fn)

  0%|          | 0/30 [00:00<?, ?it/s]

model is been trained on cuda:1
Making batches... batch size: 64,num of batchese: 782


  3%|▎         | 1/30 [00:04<02:07,  4.40s/it]

Epoch: 0, total loss: 935.3970259428024
Making batches... batch size: 64,num of batchese: 782


  7%|▋         | 2/30 [00:08<02:03,  4.40s/it]

...

Epoch: 27, total loss: 846.5800987482071
Making batches... batch size: 64,num of batchese: 782


 97%|█████████▋| 29/30 [02:07<00:04,  4.38s/it]

Epoch: 28, total loss: 851.8938454389572
Making batches... batch size: 64,num of batchese: 782


100%|██████████| 30/30 [02:11<00:00,  4.38s/it]

Epoch: 29, total loss: 841.2488782405853
Saving model to /data/gpf/gpf_dl/tests/tmp_torch_model.torch


In [20]:
img_paths, labels = zip(*path_labels_test)
dl = core.dl_img(np.array(img_paths), np.array(labels).astype(int),bs=64)
imgs,labels = zip(*dl)

preds = core.predict_batch_img(m,imgs)
expects = []
a = [expects.extend(v.tolist()) for v in labels]
core.evaluation_matrix(np.array(preds),np.array(expects))

Making batches... batch size: 64,num of batchese: 157
---------------------------0-------------------------------
total:  1000
correct:  544
accuracy:  0.544
---------------------------1-------------------------------
total:  1000
correct:  467
accuracy:  0.467
---------------------------2-------------------------------
total:  1000
correct:  341
accuracy:  0.341
---------------------------3-------------------------------
total:  1000
correct:  555
accuracy:  0.555
---------------------------4-------------------------------
total:  1000
correct:  248
accuracy:  0.248
---------------------------5-------------------------------
total:  1000
correct:  453
accuracy:  0.453
---------------------------6-------------------------------
total:  1000
correct:  538
accuracy:  0.538
---------------------------7-------------------------------
total:  1000
correct:  324
accuracy:  0.324
---------------------------8-------------------------------
total:  1000
correct:  450
accuracy:  0.45
-----------

In [21]:
conv_m = model.ConvNet([3,20,40,80],10).cuda()
conv_m

ConvNet(
  (convlayers): ModuleList(
    (0): Conv2d(3, 20, kernel_size=(3, 3), stride=(2, 2))
    (1): Conv2d(20, 40, kernel_size=(3, 3), stride=(2, 2))
    (2): Conv2d(40, 80, kernel_size=(3, 3), stride=(2, 2))
  )
  (pool): AdaptiveMaxPool2d(output_size=1)
  (output): Linear(in_features=80, out_features=10, bias=True)
)

In [25]:
opt = optim.SGD(conv_m.parameters(),lr=0.01,momentum=0.5)
loss_fn = F.nll_loss

In [28]:
img_paths, labels = zip(*path_labels_train)
dl = core.dl_img(np.array(img_paths), np.array(labels).astype(int),bs=64)
core.fit(conv_m,dl,30,opt,loss_fn)

  0%|          | 0/30 [00:00<?, ?it/s]

model is been trained on cuda:1
Making batches... batch size: 64,num of batchese: 782


  3%|▎         | 1/30 [00:04<02:22,  4.91s/it]

Epoch: 0, total loss: 183.50784113258123
Making batches... batch size: 64,num of batchese: 782


  7%|▋         | 2/30 [00:09<02:18,  4.94s/it]

...

Epoch: 27, total loss: 104.42036105692387
Making batches... batch size: 64,num of batchese: 782


 97%|█████████▋| 29/30 [02:24<00:04,  5.00s/it]

Epoch: 28, total loss: 102.62185747176409
Making batches... batch size: 64,num of batchese: 782


100%|██████████| 30/30 [02:29<00:00,  5.00s/it]

Epoch: 29, total loss: 100.39600658416748
Saving model to /data/gpf/gpf_dl/tests/tmp_torch_model.torch


In [29]:
img_paths, labels = zip(*path_labels_test)
dl = core.dl_img(np.array(img_paths), np.array(labels).astype(int),bs=64)
imgs,labels = zip(*dl)

preds = core.predict_batch_img(conv_m,imgs)
expects = []
a = [expects.extend(v.tolist()) for v in labels]
core.evaluation_matrix(np.array(preds),np.array(expects))

Making batches... batch size: 64,num of batchese: 157
---------------------------0-------------------------------
total:  1000
correct:  601
accuracy:  0.601
---------------------------1-------------------------------
total:  1000
correct:  640
accuracy:  0.64
---------------------------2-------------------------------
total:  1000
correct:  482
accuracy:  0.482
---------------------------3-------------------------------
total:  1000
correct:  707
accuracy:  0.707
---------------------------4-------------------------------
total:  1000
correct:  454
accuracy:  0.454
---------------------------5-------------------------------
total:  1000
correct:  588
accuracy:  0.588
---------------------------6-------------------------------
total:  1000
correct:  698
accuracy:  0.698
---------------------------7-------------------------------
total:  1000
correct:  371
accuracy:  0.371
---------------------------8-------------------------------
total:  1000
correct:  616
accuracy:  0.616
-----------